> Summarized from this awesome video about the discussion of chains and agents: https://www.youtube.com/watch?v=bYLHklxEd_k&t=1801s


- Chains and agents represent two concepts in technology with distinct functionalities. Chains refer to linear sequences of events or processes that rely on the step-by-step execution of tasks, often requiring predefined inputs and producing predictable outcomes. 

- Agents, conversely, are autonomous systems capable of making decisions and performing tasks independently, adapting to new situations without explicit instructions. While chains excel in structured environments with clear rules, agents are designed to navigate complex, dynamic scenarios, learning and adjusting their behavior over time. The core difference lies in the level of autonomy and complexity each can handle, with agents typically seen as more advanced due to their self-governing nature.

Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

In [ ]:
# !pip install ipydrawio
# !pip install ipydrawio_widgets

![](2023-10-27-19-28-10.png)

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [ ]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
from langchain.tools.render import format_tool_to_openai_function
llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

In [ ]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [ ]:
agent.invoke({
    "input": "how many letters in the word educa?",
    "intermediate_steps": []
})

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'educa'}, log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_word_length', 'arguments': '{\n  "word": "educa"\n}'}})])

In [ ]:
from langchain.schema.agent import AgentFinish
intermediate_steps = []
while True:
    output = agent.invoke({
        "input": "how many letters in the word educa?",
        "intermediate_steps": intermediate_steps
    })
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(output.tool, output.tool_input)
        tool = {
            "get_word_length": get_word_length
        }[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

get_word_length {'word': 'educa'}
There are 5 letters in the word "educa".


In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "how many letters in the word educa?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'how many letters in the word educa?',
 'output': 'There are 5 letters in the word "educa".'}

In [ ]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
from langchain.schema.messages import HumanMessage, AIMessage
chat_history = []

In [ ]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.append(HumanMessage(content=input1))
chat_history.append(AIMessage(content=result['output']))
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'No, "educa" is not a real word in English.'}

# Version 2 (revised?)

Steps for building a simple agent:

- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
# setup llm
llm = ChatOpenAI(temperature=0)

In [ ]:
# define tool
from langchain.tools import tool
import os
import nbformat

In [ ]:
@tool
def turn_jupyter_notebook_into_python_script(jupyter_notebook_path: str) -> None:
        """Turn a jupyter notebook into a python script or a text file"""
        # Load the notebook
        output_file_path="./output_notebook.txt" 
        file_type='txt'
        with open(jupyter_notebook_path) as f:
            nb = nbformat.read(f, as_version=4)
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
        
        # Convert the notebook to a Python script or a text file
        if file_type == 'py':
            nbformat.write(nb, output_file_path)
        elif file_type == 'txt':
            with open(output_file_path, 'w') as f:
                for cell in nb.cells:
                    if cell.cell_type == 'code':
                        f.write(cell.source + '\n\n')
                    elif cell.cell_type == 'markdown':
                        f.write(cell.source + '\n\n')
        else:
            raise ValueError('Invalid file type. Must be "py" or "txt".')
        
        
tools = [turn_jupyter_notebook_into_python_script]

In [ ]:
# setup prompt template

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant that helps me write reports for machine learning related tasks."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
from langchain.tools.render import format_tool_to_openai_function

In [ ]:
# connect llm with your tools

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [ ]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
agent = {"input": lambda x: x["input"],
         "agent_scratchpad": lambda x: format_to_openai_functions(x["intermediate_steps"]),
         } | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser() 

In [ ]:
agent.invoke({
    "input": "I want to write a report from this notebook I've written here: ./reporting_agent_langchain.ipynb",
    "intermediate_steps": []})

AgentActionMessageLog(tool='turn_jupyter_notebook_into_python_script', tool_input={'jupyter_notebook_path': './reporting_agent_langchain.ipynb'}, log="\nInvoking: `turn_jupyter_notebook_into_python_script` with `{'jupyter_notebook_path': './reporting_agent_langchain.ipynb'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'turn_jupyter_notebook_into_python_script', 'arguments': '{\n  "jupyter_notebook_path": "./reporting_agent_langchain.ipynb"\n}'}})])

In [ ]:
from langchain.schema.agent import AgentFinish